In [18]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
! pip install geocoder
! pip install --upgrade geocoder
import geocoder

  Attempting uninstall: geocoder
    Found existing installation: geocoder 0.6.0
    Uninstalling geocoder-0.6.0:
      Successfully uninstalled geocoder-0.6.0


In [2]:
wiki_link = 'https://en.wikipedia.org/wiki/Ankara_Province'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:64.0) Gecko/20100101 Firefox/64.0'}
wiki_page = requests.get(wiki_link, headers = headers)
wiki_page

<Response [200]>

In [3]:
soup = BeautifulSoup(wiki_page.content, 'html.parser')
table = soup.find('table', {'class':'wikitable sortable'}).tbody

In [4]:
rows = table.find_all('tr')

columns = [i.text.replace('\n', '') for i in rows[0].find_all('th')]
columns

['District', 'Population (2017)', 'Area (km²)', 'Density (per km²)']

In [5]:
df_ankara = pd.DataFrame(columns = columns)

In [7]:

for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df_ankara = df_ankara.append(pd.Series(values, index = columns), ignore_index = True)

        df_ankara

In [8]:
df_ankara.head()

,District,Population (2017),Area (km²),Density (per km²)
0,Akyurt,32.863,369,89
1,Altındağ,371.366,123,3.019
2,Ayaş,12.289,1.041,12
3,Bala,21.682,1.851,12
4,Beypazarı,48.476,1.697,29


In [20]:
def get_latlng(arcgis_geocoder):
    
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Ankara, Turkey'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [21]:
districts = df_ankara['District']    
coordinates = [get_latlng(districts) for districts in districts.tolist()]

In [22]:
df_ankara_loc = df_ankara

df_ankara_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df_ankara_loc['Latitude'] = df_ankara_coordinates['Latitude']
df_ankara_loc['Longitude'] = df_ankara_coordinates['Longitude']

In [23]:
df_ankara_loc.drop(columns="Population (2017)", axis=1, inplace=True)
df_ankara_loc.drop(columns="Density (per km²)", axis=1, inplace=True)
df_ankara_loc.drop(columns="Area (km²)", axis=1, inplace=True)
df_ankara_loc.head()


,District,Latitude,Longitude
0,Akyurt,40.13082,33.08719
1,Altındağ,39.94171,32.85445
2,Ayaş,40.01516,32.33270
3,Bala,39.55391,33.12352
4,Beypazarı,40.16811,31.92052


In [24]:
import numpy as np
import json 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium


Libraries imported


In [25]:
from geopy.geocoders import Nominatim 

address = "Çankaya, Ankara"

geolocator = Nominatim(user_agent = "Ankara_explorer")

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("The geographical coordinates of Ankara are {}, {}.".format(latitude, longitude))

The geographical coordinates of Ankara are 39.9207893, 32.8540412.


In [26]:
map_ankara = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, label in zip(df_ankara_loc["Latitude"], df_ankara_loc["Longitude"], df_ankara_loc["District"]):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=25,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_ankara)  
    
map_ankara

In [27]:
CLIENT_ID = "FBMIJHTR42BI0I5JUWFNREL3NTHT553IQUQ3AM1NGMQDDSHJ"
CLIENT_SECRET = "OVH1MFLGAQG4O4IKBKKHQLZJTDBL2QNFSV2R1NKA4YUBINQC" 
VERSION = "20180605"

In [28]:
df_ankara_loc.loc [0, "District"]
df_ankara_loc.loc [0, "District"]

'Akyurt'

In [29]:
neighborhood_latitude = df_ankara_loc.loc[0, "Latitude"]
neighborhood_longitude = df_ankara_loc.loc[0, "Longitude"] 

neighborhood_name = df_ankara_loc.loc[0, "District"] 

print("Latitude and longitude values of the neighborhood {} are {}, {}.".format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and longitude values of the neighborhood Akyurt are 40.13082000000003, 33.08719000000008.


In [30]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=FBMIJHTR42BI0I5JUWFNREL3NTHT553IQUQ3AM1NGMQDDSHJ&client_secret=OVH1MFLGAQG4O4IKBKKHQLZJTDBL2QNFSV2R1NKA4YUBINQC&v=20180605&ll=40.13082000000003,33.08719000000008&radius=500&limit=100'

In [31]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6108812fc4a8dc6fed8c0996'},
 'response': {'headerLocation': 'Akyurt',
  'headerFullLocation': 'Akyurt, Ankara',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 16,
  'suggestedBounds': {'ne': {'lat': 40.13532000450003,
    'lng': 33.093064640939495},
   'sw': {'lat': 40.126319995500026, 'lng': 33.08131535906066}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5049c781e4b0a7a32d840eb1',
       'name': 'Meşhur Köfteci Bodur',
       'location': {'address': 'Cumhuriyet Meydani',
        'crossStreet': 'Cumhuriyet Meydani',
        'lat': 40.13071902888477,
        'lng': 33.08677622815274,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.13071902888477,
          'lng': 33.08677622815274}],
        '

In [32]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)

<ipython-input-33-f5e18700a056>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Meşhur Köfteci Bodur,Turkish Restaurant,40.130719,33.086776
1,Akyurt kültür parkı,Park,40.133123,33.085071
2,Yurtalan Restaurant,Kebab Restaurant,40.132672,33.084306
3,Onur pide ve kebap salonu,Kebab Restaurant,40.130807,33.086422
4,Beyazıt Sofrası,Steakhouse,40.130542,33.081901
5,Yurtalan,Turkish Restaurant,40.132743,33.084385
6,Yurtalan Restaurant,Doner Restaurant,40.132641,33.084492
7,Benliler Supermarket,Convenience Store,40.130252,33.081423
8,şişman pasta&cafe,Café,40.130820,33.085025
9,Akyurt Sofrası,Turkish Restaurant,40.130854,33.086709


In [34]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

16 venues were returned by Foursquare.


In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:

ankara_venues = getNearbyVenues(names=df_ankara_loc['District'],
                                   latitudes=df_ankara_loc['Latitude'],
                                   longitudes=df_ankara_loc['Longitude']
                                  )

Akyurt
Altındağ
Ayaş
Bala
Beypazarı
Çamlıdere
Çankaya
Çubuk
Elmadağ
Etimesgut
Evren
Gölbaşı
Güdül
Haymana
Kahramankazan
Kalecik
Keçiören
Kızılcahamam
Mamak
Nallıhan
Polatlı
Pursaklar
Sincan
Şereflikoçhisar
Yenimahalle
Urban (9 districts)Altındağ, Çankaya, Etimesgut, Gölbaşı, Keçiören, Mamak, Pursaklar, Sincan, Yenimahalle
TOTAL


In [37]:
print(ankara_venues.shape)
ankara_venues.head(250)

(1691, 7)


,District,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Akyurt,40.13082,33.08719,Akyurt kültür parkı,40.133123,33.085071,Park
1,Akyurt,40.13082,33.08719,Meşhur Köfteci Bodur,40.130719,33.086776,Turkish Restaurant
2,Akyurt,40.13082,33.08719,Yurtalan Restaurant,40.132672,33.084306,Kebab Restaurant
3,Akyurt,40.13082,33.08719,Beyazıt Sofrası,40.130542,33.081901,Steakhouse
4,Akyurt,40.13082,33.08719,Onur pide ve kebap salonu,40.130807,33.086422,Kebab Restaurant
...,...,...,...,...,...,...,...
245,Beypazarı,40.16811,31.92052,Beypazarı Boztepe,40.172056,31.930631,Mountain
246,Beypazarı,40.16811,31.92052,Aras Eczanesi,40.162724,31.930814,Pharmacy
247,Beypazarı,40.16811,31.92052,Yavaşzade Optik,40.163193,31.931954,Optical Shop
248,Beypazarı,40.16811,31.92052,KASIMIN YERİ,40.172880,31.932553,Falafel Restaurant


In [38]:
ankara_venues.groupby("District").count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Akyurt,29,29,29,29,29,29
Altındağ,100,100,100,100,100,100
Ayaş,39,39,39,39,39,39
Bala,16,16,16,16,16,16
Beypazarı,71,71,71,71,71,71
Elmadağ,52,52,52,52,52,52
Etimesgut,100,100,100,100,100,100
Evren,5,5,5,5,5,5
Gölbaşı,100,100,100,100,100,100


In [39]:
print('There are {} unique venue categories.'.format(len(ankara_venues['Venue Category'].unique())))

There are 226 unique venue categories.


In [40]:
ankara_onehot = pd.get_dummies(ankara_venues[['Venue Category']], prefix="", prefix_sep="")

ankara_onehot['Neighborhood'] = ankara_venues['District'] 

fixed_columns = [ankara_onehot.columns[-1]] + list(ankara_onehot.columns[:-1])
ankara_onehot = ankara_onehot[fixed_columns]

ankara_onehot

,Neighborhood,ATM,Advertising Agency,Airport Lounge,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,...,Turkish Restaurant,Used Bookstore,Video Game Store,Vineyard,Volleyball Court,Water Park,Waterfront,Wedding Hall,Women's Store,Yoga Studio
0,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Akyurt,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1686,TOTAL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1687,TOTAL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1688,TOTAL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1689,TOTAL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
ankara_onehot.shape

(1691, 227)

In [42]:
ankara_grouped = ankara_onehot.groupby('Neighborhood').mean().reset_index()
ankara_grouped

,Neighborhood,ATM,Advertising Agency,Airport Lounge,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,...,Turkish Restaurant,Used Bookstore,Video Game Store,Vineyard,Volleyball Court,Water Park,Waterfront,Wedding Hall,Women's Store,Yoga Studio
0,Akyurt,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.103448,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.034483,0.00
1,Altındağ,0.000000,0.00,0.000000,0.03,0.000000,0.000000,0.02,0.020000,0.000000,...,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00
2,Ayaş,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.128205,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00
3,Bala,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.062500,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00
4,Beypazarı,0.000000,0.00,0.000000,0.00,0.000000,0.014085,0.00,0.000000,0.014085,...,0.070423,0.00,0.000000,0.000000,0.014085,0.000000,0.0,0.000000,0.000000,0.00
5,Elmadağ,0.019231,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.019231,0.00,0.000000,0.000000,0.000000,0.019231,0.0,0.000000,0.000000,0.00
6,Etimesgut,0.000000,0.00,0.000000,0.00,0.000000,0.040000,0.00,0.000000,0.000000,...,0.090000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.01
7,Evren,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.00
8,Gölbaşı,0.000000,0.00,0.000000,0.00,0.000000,0.040000,0.00,0.000000,0.000000,...,0.100000,0.00,0.000000,0.000000,0.000000,0.010000,0.0,0.000000,0.000000,0.00
9,Güdül,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.111111,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00


In [43]:
ankara_grouped.shape

(27, 227)

In [44]:
num_top_venues = 10

for hood in ankara_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ankara_grouped[ankara_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Akyurt----
                venue  freq
0                Farm  0.14
1  Turkish Restaurant  0.10
2    Kebab Restaurant  0.07
3    Botanical Garden  0.07
4                Park  0.07
5    Basketball Court  0.03
6            Pharmacy  0.03
7        Camera Store  0.03
8             Stadium  0.03
9          Steakhouse  0.03


----Altındağ----
                venue  freq
0                Café  0.10
1      History Museum  0.07
2               Hotel  0.06
3       Historic Site  0.05
4  Turkish Restaurant  0.04
5          Restaurant  0.04
6             Theater  0.04
7       Jewelry Store  0.04
8         Coffee Shop  0.03
9          Steakhouse  0.03


----Ayaş----
                     venue  freq
0       Turkish Restaurant  0.13
1                     Park  0.08
2                     Lake  0.05
3                     Café  0.05
4               Restaurant  0.05
5  Fruit & Vegetable Store  0.03
6                 Mountain  0.03
7            Deli / Bodega  0.03
8                 Tea Room  0.03
9    

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
ankara_venues_sorted = pd.DataFrame(columns=columns)
ankara_venues_sorted['Neighborhood'] = ankara_grouped['Neighborhood']

for ind in np.arange(ankara_grouped.shape[0]):
    ankara_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ankara_grouped.iloc[ind, :], num_top_venues)

ankara_venues_sorted.head(27)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akyurt,Farm,Turkish Restaurant,Kebab Restaurant,Botanical Garden,Park,Big Box Store,Camera Store,Music Venue,Gym,Steakhouse
1,Altındağ,Café,History Museum,Hotel,Historic Site,Turkish Restaurant,Restaurant,Theater,Jewelry Store,Antique Shop,Furniture / Home Store
2,Ayaş,Turkish Restaurant,Park,Lake,Café,Restaurant,Lounge,Taxi Stand,Coffee Shop,Erotic Museum,Steakhouse
3,Bala,Park,Electronics Store,Mountain,Café,Business Service,Bus Station,Shopping Mall,Mobile Phone Shop,Nature Preserve,Turkish Restaurant
4,Beypazarı,Turkish Restaurant,Restaurant,History Museum,Bakery,Tea Room,Scenic Lookout,Motel,Bed & Breakfast,Park,Hotel
5,Elmadağ,Café,Park,Campground,Convenience Store,Bar,Kebab Restaurant,Dessert Shop,Seafood Restaurant,Bus Stop,Restaurant
6,Etimesgut,Turkish Restaurant,Café,Restaurant,Arcade,Steakhouse,Gym,Shopping Mall,Park,Kebab Restaurant,Dessert Shop
7,Evren,Park,Waterfront,Steakhouse,River,Yoga Studio,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Flower Shop
8,Gölbaşı,Café,Turkish Restaurant,Park,Restaurant,Coffee Shop,Arcade,Breakfast Spot,Dessert Shop,Steakhouse,Middle Eastern Restaurant
9,Güdül,Plaza,Department Store,Restaurant,Botanical Garden,Castle,Turkish Restaurant,Souvenir Shop,Moving Target,Food Truck,Food Court


In [54]:
from sklearn.cluster import KMeans


In [58]:
ankara_merged = df_ankara
ankara_merged['Cluster Labels'] = KMeans

ankara_merged = ankara_merged.join(ankara_venues_sorted.set_index('Neighborhood'), on='District')
ankara_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akyurt,Farm,Turkish Restaurant,Kebab Restaurant,Botanical Garden,Park,Big Box Store,Camera Store,Music Venue,Gym,Steakhouse
1,Altındağ,Café,History Museum,Hotel,Historic Site,Turkish Restaurant,Restaurant,Theater,Jewelry Store,Antique Shop,Furniture / Home Store
2,Ayaş,Turkish Restaurant,Park,Lake,Café,Restaurant,Lounge,Taxi Stand,Coffee Shop,Erotic Museum,Steakhouse
3,Bala,Park,Electronics Store,Mountain,Café,Business Service,Bus Station,Shopping Mall,Mobile Phone Shop,Nature Preserve,Turkish Restaurant
4,Beypazarı,Turkish Restaurant,Restaurant,History Museum,Bakery,Tea Room,Scenic Lookout,Motel,Bed & Breakfast,Park,Hotel
5,Elmadağ,Café,Park,Campground,Convenience Store,Bar,Kebab Restaurant,Dessert Shop,Seafood Restaurant,Bus Stop,Restaurant
6,Etimesgut,Turkish Restaurant,Café,Restaurant,Arcade,Steakhouse,Gym,Shopping Mall,Park,Kebab Restaurant,Dessert Shop
7,Evren,Park,Waterfront,Steakhouse,River,Yoga Studio,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Flower Shop
8,Gölbaşı,Café,Turkish Restaurant,Park,Restaurant,Coffee Shop,Arcade,Breakfast Spot,Dessert Shop,Steakhouse,Middle Eastern Restaurant
9,Güdül,Plaza,Department Store,Restaurant,Botanical Garden,Castle,Turkish Restaurant,Souvenir Shop,Moving Target,Food Truck,Food Court


In [59]:

ks = 3

ankara_grouped_clustering = ankara_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=ks, random_state=0).fit(ankara_grouped_clustering)

kmeans.labels_[1:10]

array([1, 1, 1, 1, 1, 1, 0, 1, 2])

In [63]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(ks)
ys = [i+x+(i*x)**2 for i in range(ks)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ankara_merged['Latitude'], ankara_merged['Longitude'], ankara_merged['District'], ankara_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow,
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters